In [1]:
import os

os.environ['R_HOME'] = '/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/R'

In [2]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


In [3]:
sc_ad = sc.read_h5ad('/home/comp/cszrwang/project/Geneformer_test/STARmap/toy/starmap_sc_rna.h5ad')
st_ad = sc.read_h5ad('/home/comp/cszrwang/project/Geneformer_test/STARmap/toy/starmap_spatial.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [4]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
celltype
Excitatory L2/3     200
Excitatory L6       200
Excitatory L5       197
Excitatory L4       190
Inhibitory Other    185
Inhibitory Sst      178
Inhibitory Pvalb    174
Inhibitory Vip      170
Astro               156
Other               120
Endo                107
Olig                 97
Smc                  94
Micro                57
Neuron Other         51
Name: count, dtype: int64
### Used gene numbers: 791
### Initializing sample probability
### 

In [5]:
spoint_model.train(max_steps=5000, batch_size=1024)

/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.29it/s, loss=1.12e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.53it/s, loss=1.12e+03, v_num=1]


Step 5000: Test inference loss=-0.954: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [14:29<00:00,  5.75it/s]


In [6]:
pre = spoint_model.deconv_spatial()

In [7]:
pre

,Excitatory L6,Excitatory L5,Inhibitory Sst,Inhibitory Vip,Excitatory L4,Inhibitory Pvalb,Inhibitory Other,Excitatory L2/3,Astro,Other,Endo,Olig,Smc,Neuron Other,Micro
0,0.018493,0.157463,0.232316,0.124256,0.069572,0.000000,0.00000,0.146563,0.235930,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,0.012688,0.148688,0.125041,0.095403,0.045554,0.011230,0.00000,0.000000,0.196703,0.000000,0.000000,0.212110,0.0,0.128060,0.000000
2,0.012105,0.141982,0.102010,0.029656,0.021296,0.080212,0.00000,0.036500,0.124397,0.000000,0.000000,0.297346,0.0,0.125063,0.012616
3,0.012672,0.065212,0.040479,0.016490,0.046567,0.107553,0.00000,0.050642,0.268064,0.013997,0.019665,0.260509,0.0,0.087453,0.000000
4,0.190349,0.192449,0.145434,0.015815,0.019646,0.000000,0.00000,0.064641,0.037834,0.000000,0.033817,0.147529,0.0,0.122577,0.016697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0.000000,0.155735,0.102002,0.000000,0.108587,0.133010,0.11568,0.159632,0.132735,0.000000,0.000000,0.043573,0.0,0.012766,0.016981
185,0.000000,0.000000,0.011519,0.076179,0.058111,0.000000,0.00000,0.440815,0.182619,0.000000,0.000000,0.015461,0.0,0.168171,0.022926
186,0.000000,0.010306,0.000000,0.189908,0.486748,0.000000,0.00000,0.000000,0.210841,0.000000,0.000000,0.013559,0.0,0.000000,0.081069
187,0.076745,0.023762,0.000000,0.183184,0.000000,0.000000,0.00000,0.000000,0.451990,0.000000,0.000000,0.245038,0.0,0.000000,0.000000


In [8]:
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/STARmap/SPACEL/"

In [9]:
pre.to_csv(output_file_path + '/SPACEL_result.csv')

In [10]:
st_ad_show = spoint_model.st_ad

In [11]:
st_ad_show

AnnData object with n_obs × n_vars = 189 × 791
    obs: 'n_genes', 'batch', 'Excitatory L6', 'Excitatory L5', 'Inhibitory Sst', 'Inhibitory Vip', 'Excitatory L4', 'Inhibitory Pvalb', 'Inhibitory Other', 'Excitatory L2/3', 'Astro', 'Other', 'Endo', 'Olig', 'Smc', 'Neuron Other', 'Micro'
    var: 'n_cells'
    uns: 'celltypes'
    layers: 'norm'

In [12]:
sc.pl.embedding(st_ad_show,color=pre.columns[:20],basis='spatial',ncols=5)

KeyError: "Could not find 'spatial' or 'X_spatial' in .obsm"